# TAREA LOG NASA EN SQL 

* Autor: Beltrán Aller López
* Fecha: 02/12/2019
* Asignatura: ETL

En primer lugar se genera el contexto spark.

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

Procedo a la carga del archivo, utilizaré la base de datos pequeña, por lo que los resultados serán distintos.

In [3]:
data_file = "./apache.access.log_small"
raw_data = sc.textFile(data_file)

Realizo un conteo para comprobar que se ha cargado bien.

In [4]:
raw_data.count()

3432

Extraigo una muestra para visualizar el formato que tiene.

In [5]:
raw_data.take(4)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0']

Observo lo siguiente:

* En primer lugar hay una cadena de caractéres.
* Luego dos espacios en blanco y dos guiones.
* Entre corchetes la fecha en formato: dd/MM/yyyy:hh:mm:ss y un guión con 4 números.
* Se observa un espacio en blanco justo después de la fecha.
* Comienza una cadena de caractéres entre comillas y separados por /,- y un espacio en blanco al principio y otro al final.
* Espacio en blanco, y dos números de distinta longitud, el primero para el código y el segundo para el tamaño.

Guardo una de las muestras para poner a prueba la expresión regular.

In [6]:
prueba = raw_data.take(1)

In [7]:
prueba[0]

'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'

Importo el paquete de expresiones regulares.

In [8]:
import re

Defino mi expresión regular.

In [9]:
expresion = '^(\S+)\s\-\s\- \[(\S+) ([-]\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w+\.\s*]+)"* (\d{3}) (\S+)'

La pongo a prueba.

In [10]:
re.search(expresion, prueba[0]).groups()

('in24.inetnebr.com',
 '01/Aug/1995:00:00:01',
 '-0400',
 'GET',
 '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
 'HTTP/1.',
 '0',
 '200',
 '1839')

Defino la función de parseado, parseo el archivo y también defino una función para convertir a entero el tamaño.

In [11]:
def map_log(line):
    match = re.search(expresion, line)
    return match.groups()

In [12]:
parse_def = raw_data.map(lambda x: map_log(x))

In [13]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)

Creo el contexto SQL e importo la librería Row para SQL y datetime para las fechas.

In [14]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [15]:
from pyspark.sql import Row 

In [16]:
from datetime import datetime

In [17]:
base_datos = parse_def.map(lambda p: Row(
    host=p[0], 
    fecha=datetime.strptime(p[1][:11], "%d/%b/%Y"),
    endpoint=p[4],
    codigo=p[7],
    size=convert_long(p[8])
    )
)

In [18]:
lognasa_df = sqlContext.createDataFrame(base_datos)
lognasa_df.registerTempTable("NASA")

### Ejercicio 1: Mínimo, máximo y media del tamaño de las peticiones 

In [19]:
stats = sqlContext.sql("""
SELECT MAX(size), MIN(size), AVG(size)
FROM NASA
""")
stats.show()

+---------+---------+------------------+
|max(size)|min(size)|         avg(size)|
+---------+---------+------------------+
|   887988|        0|16051.863636363636|
+---------+---------+------------------+



### Ejercicio 2: Nº de peticiones de cada código respuesta

In [20]:
num_peticiones = sqlContext.sql("""
SELECT codigo, COUNT(codigo)
FROM NASA
GROUP BY codigo
""")
num_peticiones.show()

+------+-------------+
|codigo|count(codigo)|
+------+-------------+
|   200|         3140|
|   302|           50|
|   404|           22|
|   403|            1|
|   304|          219|
+------+-------------+



### Ejercicio 3: Mostrar 20 hosts que han sido visitados más de 10 veces

In [21]:
hosts = sqlContext.sql("""
SELECT host, COUNT(host)
FROM NASA
GROUP BY host
HAVING COUNT(host) > 10
LIMIT 20
""")
hosts.show()

+--------------------+-----------+
|                host|count(host)|
+--------------------+-----------+
|     info.telenor.no|         15|
|in2pc2.med.niigat...|         14|
|ix-sf10-28.ix.net...|         12|
|   in24.inetnebr.com|         55|
|     204.199.188.113|         12|
|     pm6a3.sover.net|         11|
|www-b5.proxy.aol.com|         28|
|ix-min1-02.ix.net...|         78|
|pppa006.compuserv...|         17|
|ad14-027.compuser...|         16|
|term1-24.sb.west.net|         29|
|       193.84.66.147|         31|
| mac19.ils.uec.ac.jp|         14|
|       133.68.18.180|         17|
|      139.230.35.135|         17|
|haraway.ucet.ufl.edu|         32|
|ad11-061.compuser...|         13|
|hsccs_gatorbox07....|         40|
|  cs1-06.leh.ptd.net|         19|
|violin-05.synapse...|         17|
+--------------------+-----------+



### Ejercicio 4: Mostrar los 10 endpoints más visitados

In [22]:
endpoint = sqlContext.sql("""
SELECT endpoint, COUNT(endpoint)
FROM NASA
GROUP BY endpoint
ORDER BY COUNT(endpoint) DESC
LIMIT 10
""")
endpoint.show()

+--------------------+---------------+
|            endpoint|count(endpoint)|
+--------------------+---------------+
|/images/KSC-logos...|            167|
|/images/NASA-logo...|            159|
|/images/MOSAIC-lo...|            121|
|/images/WORLD-log...|            119|
|/images/USA-logos...|            117|
|/images/ksclogo-m...|            105|
|                   /|             84|
|/history/apollo/i...|             74|
|/images/launch-lo...|             69|
|/images/ksclogosm...|             66|
+--------------------+---------------+



### Ejercicio 5: Mostrar los 10 endpoints más visitados que no tienen código de respuesta =200

In [23]:
endpoint = sqlContext.sql("""
SELECT endpoint, COUNT(endpoint)
FROM NASA
WHERE codigo != 200
GROUP BY endpoint
ORDER BY COUNT(endpoint) DESC
LIMIT 10
""")
endpoint.show()

+--------------------+---------------+
|            endpoint|count(endpoint)|
+--------------------+---------------+
|/images/NASA-logo...|             25|
|/images/KSC-logos...|             24|
|/images/MOSAIC-lo...|             17|
|/images/WORLD-log...|             17|
|/images/USA-logos...|             16|
|/images/ksclogo-m...|             10|
|/software/winvn/b...|              8|
|/images/construct...|              8|
|/software/winvn/w...|              8|
|/software/winvn/w...|              6|
+--------------------+---------------+



### Ejercicio 6:  Calcular el nº de hosts distintos

In [24]:
hosts_distintos = sqlContext.sql("""
SELECT COUNT(DISTINCT host)
FROM NASA
""")
hosts_distintos.show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|                 311|
+--------------------+



__NOTA:__ El número de host distintos en la base de datos grandes será otro.

### Ejercicio 7: Contar el nº de hosts únicos cada día

In [25]:
hosts_unicos = sqlContext.sql("""
SELECT fecha, COUNT(DISTINCT host)
FROM NASA
GROUP BY fecha
""")
hosts_unicos.show()

+--------------------+--------------------+
|               fecha|count(DISTINCT host)|
+--------------------+--------------------+
|1995-08-01 00:00:...|                 311|
+--------------------+--------------------+



__NOTA__: Al trabajar con la base de datos pequeña, se disponde de un único día.

### Ejercicio 8: Calcular la media de peticiones diarias 

In [26]:
peticiones = sqlContext.sql("""
SELECT fecha, (COUNT(host) / COUNT(DISTINCT fecha)) AS media_peticiones_diarias 
FROM NASA
GROUP BY fecha
""")
peticiones.show()

+--------------------+------------------------+
|               fecha|media_peticiones_diarias|
+--------------------+------------------------+
|1995-08-01 00:00:...|                  3432.0|
+--------------------+------------------------+



__NOTA:__ Sólo tenemos un día al estar usando la base de datos pequeña. Por tanto, la media en este caso será el número de peticiones que tenemos este día.

### Ejercicio 9: Mostrar una lista de 40 endpoints distintos que generan código de respuesta =404 

In [27]:
endpoint_404 = sqlContext.sql("""
SELECT DISTINCT(endpoint)
FROM NASA
WHERE codigo = 404
LIMIT 40
""")
endpoint_404.show()

+--------------------+
|            endpoint|
+--------------------+
|/pub/winvn/readme...|
|/elv/DELTA/uncons...|
|/history/apollo/a...|
|/history/apollo/a...|
|       /sts-71/launc|
|/history/apollo/a...|
|/history/apollo/a...|
|/shuttle/resource...|
|/www/software/win...|
|/pub/winvn/releas...|
|/history/apollo/a...|
|/history/apollo/a...|
|/history/history.htm|
+--------------------+



__NOTA:__ El resultado esá condicionado por la base de datos.

### Ejercicio 10: Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan

In [28]:
endpoint_top25 = sqlContext.sql("""
SELECT endpoint, COUNT(codigo)
FROM NASA
WHERE codigo = 404
GROUP BY endpoint
ORDER BY COUNT(codigo) DESC
LIMIT 25
""")
endpoint_top25.show()

+--------------------+-------------+
|            endpoint|count(codigo)|
+--------------------+-------------+
|/pub/winvn/releas...|            4|
|/history/apollo/a...|            4|
|/history/apollo/a...|            2|
|/history/apollo/a...|            2|
|/pub/winvn/readme...|            2|
|/elv/DELTA/uncons...|            1|
|       /sts-71/launc|            1|
|/history/apollo/a...|            1|
|/shuttle/resource...|            1|
|/history/apollo/a...|            1|
|/history/apollo/a...|            1|
|/www/software/win...|            1|
|/history/history.htm|            1|
+--------------------+-------------+



__NOTA:__ El resultado esá condicionado por la base de datos.

### Ejercicio 11: El top 5 de días que se generaron código de respuestas 404

In [29]:
date_top5 = sqlContext.sql("""
SELECT fecha, COUNT(codigo)
FROM NASA
WHERE codigo = 404
GROUP BY fecha
ORDER BY COUNT(codigo) DESC
LIMIT 5
""")
date_top5.show()

+--------------------+-------------+
|               fecha|count(codigo)|
+--------------------+-------------+
|1995-08-01 00:00:...|           22|
+--------------------+-------------+



__NOTA:__ Sólo tenemos un día al estar operando con la base de datos pequeña.